<a href="https://colab.research.google.com/github/Nadia147/Machine_learning-Deep_learning/blob/main/change_win_3dcnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install earthpy
# Colab Requirements
!pip install spectral
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
## Basics
import gc
gc.collect()
import warnings
warnings.filterwarnings('ignore')
import time
import numpy
import numpy as np
from PIL import Image
from operator import truediv
import scipy.io as sio
import os
import pandas as pd
import seaborn as sns
import spectral
## Ploting
from plotly.offline import init_notebook_mode
import plotly.graph_objs as go
from matplotlib.pyplot import cm
import matplotlib.pyplot as plt
init_notebook_mode(connected=True)
%matplotlib inline
## Sklearn
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, cohen_kappa_score
from sklearn.decomposition import IncrementalPCA
## Deep Model
import keras
import h5py
from keras.layers import Dropout, Input, Conv2D, Conv3D, MaxPool3D, Flatten, Dense, Reshape, BatchNormalization
from plotly.offline import iplot, init_notebook_mode
from keras.losses import categorical_crossentropy
from keras.optimizers import Adadelta
from keras.models import Sequential, Model
from keras.utils import np_utils
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
## Mounting Colab
from google.colab import drive
from google.colab import files
drive.mount('/content/drive')
import matplotlib.pyplot as plt

import numpy as np
import earthpy.plot as ep
import seaborn as sns
import earthpy.spatial as es

import plotly.graph_objects as go
import plotly.express as px

from scipy.io import loadmat

import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import (accuracy_score,
                             confusion_matrix, classification_report)

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from tqdm import tqdm
import scipy.io

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.9/212.9 kB 1.8 MB/s eta 0:00:00


Instructions for updating:
non-resource variables are not supported in the long term


Mounted at /content/drive


In [ ]:

## Split Training, Validation and Test Sets
def SplitTr_Te(HSI, GT, TeRatio, randomState=345):
    Tr, Te, TrC, TeC = train_test_split(HSI, GT, test_size=TeRatio, random_state=randomState, stratify=GT)
    return Tr, Te, TrC, TeC
def DL_Method(HSI, numComponents = 75):
    RHSI = np.reshape(HSI, (-1, HSI.shape[2]))
    n_batches = 256
    inc_pca = IncrementalPCA(n_components=numComponents)
    for X_batch in np.array_split(RHSI, n_batches):
        inc_pca.partial_fit(X_batch)
    X_ipca = inc_pca.transform(RHSI)
    RHSI = np.reshape(X_ipca, (HSI.shape[0],HSI.shape[1], numComponents))
    return RHSI
## Padding
def ZeroPad(HSI, margin=2):
    NHSI = np.zeros((HSI.shape[0] + 2 * margin, HSI.shape[1] + 2* margin, HSI.shape[2]))
    x_offset = margin
    y_offset = margin
    NHSI[x_offset:HSI.shape[0] + x_offset, y_offset:HSI.shape[1] + y_offset, :] = HSI
    return NHSI
## Spatial Patches in 3D
def HSICubes(HSI, GT, WinSize=5, removeZeroLabels = True):#5t01
    margin = int((WinSize - 1) / 2)
    zeroPaddedX = ZeroPad(HSI, margin=margin)
    # split patches
    patchesData = np.zeros((HSI.shape[0] * HSI.shape[1], WinSize, WinSize, HSI.shape[2]))
    patchesLabels = np.zeros((HSI.shape[0] * HSI.shape[1]))
    patchIndex = 0
    for r in range(margin, zeroPaddedX.shape[0] - margin):
        for c in range(margin, zeroPaddedX.shape[1] - margin):
            patch = zeroPaddedX[r - margin:r + margin + 1, c - margin:c + margin + 1]
            patchesData[patchIndex, :, :, :] = patch
            patchesLabels[patchIndex] = GT[r-margin, c-margin]
            patchIndex = patchIndex + 1
    if removeZeroLabels:
        patchesData = patchesData[patchesLabels>0,:,:,:]
        patchesLabels = patchesLabels[patchesLabels>0]
        patchesLabels -= 1
    return patchesData, patchesLabels
## Compute Per Class Accuacy form Confusion Matrix
def AA_andEachClassAccuracy(confusion_matrix):
    counter = confusion_matrix.shape[0]
    list_diag = np.diag(confusion_matrix)
    list_raw_sum = np.sum(confusion_matrix, axis=1)
    each_acc = np.nan_to_num(truediv(list_diag, list_raw_sum))
    average_acc = np.mean(each_acc)
    return each_acc, average_acc
## Assigning Class Labels and Computing the Test Accuracy
def reports(Te,TeC):
    start = time.time()
    Y_pred = model.predict(Te)
    y_pred = np.argmax(Y_pred, axis=1)
    end = time.time()
    total = end - start
    target_names = ['Alfalfa', 'Corn-notill', 'Corn-mintill', 'Corn'
                        ,'Grass-pasture', 'Grass-trees', 'Grass-pasture-mowed',
                        'Hay-windrowed', 'Oats', 'Soybean-notill', 'Soybean-mintill',
                        'Soybean-clean', 'Wheat', 'Woods', 'Buildings-Grass-Trees-Drives',
                        'Stone-Steel-Towers']
    classification = classification_report(np.argmax(TeC, axis=1), y_pred, target_names=target_names)
    oa = accuracy_score(np.argmax(TeC, axis=1), y_pred)
    confusion = confusion_matrix(np.argmax(TeC, axis=1), y_pred)
    each_acc, aa = AA_andEachClassAccuracy(confusion)
    kappa = cohen_kappa_score(np.argmax(TeC, axis=1), y_pred)
    score = model.evaluate(Te, TeC, batch_size=32)
    Test_Loss =  score[0]*100
    Test_accuracy = score[1]*100
    return classification, confusion, Test_Loss, Test_accuracy, oa*100, each_acc*100, aa*100, kappa*100, target_names, y_pred, total
## Compute the Patch to Prepare for Ground Truths
def Patch(data,height_index,width_index):
    height_slice = slice(height_index, height_index+PATCH_SIZE)
    width_slice = slice(width_index, width_index+PATCH_SIZE)
    patch = data[height_slice, width_slice, :]
    return patch
## Loading Dataset
import scipy.io
## There is a typo in the paper on page#4 Table#4, the training samples are not 10%, the reported accuracies are obtained with almost 30~40% training samples.
#data_path = os.path.join(os.getcwd(),'/content/drive/My Drive/Colab Notebooks')
#HSI = sio.loadmat(os.path.join(data_path, 'Indian_pines_corrected.mat'))['indian_pines_corrected']
HSI = scipy.io.loadmat('/content/drive/MyDrive/Colab Notebooks/Indian_pines_corrected.mat')['indian_pines_corrected'];
#GT = labels = sio.loadmat(os.path.join(data_path, 'Indian_pines_gt.mat'))['indian_pines_gt']
GT = scipy.io.loadmat('/content/drive/MyDrive/Colab Notebooks/Indian_pines_gt.mat')['indian_pines_gt']
HSI.shape, GT.shape
## Reduce the Dimensionality
HSI = DL_Method(HSI, numComponents=20)
HSI.shape
## Create Image Cubes for Model Building
HSI, GT = HSICubes(HSI, GT, WinSize=9)
HSI.shape, GT.shape
## Split Train and Test sets
Tr, Te, TrC, TeC = SplitTr_Te(HSI, GT, 0.60)
Tr.shape, Te.shape, TrC.shape, TeC.shape
## Split Train and Validation
Tr, Tv, TrC, TvC = SplitTr_Te(Tr, TrC, 0.40)
Tr.shape, Tv.shape, TrC.shape, TvC.shape

Tr = Tr.reshape(-1, 9, 9, 20, 1)
TrC = np_utils.to_categorical(TrC)
Tv = Tv.reshape(-1, 9, 9, 20, 1)
TvC = np_utils.to_categorical(TvC)
Tr.shape, TrC.shape, Tv.shape, TvC.shape

## Model Structure
## Input layer
input_layer = Input((9, 9, 20, 1))
## 2D Convolutional Layers
conv_layer1 = Conv2D(filters=128, kernel_size=(1,1), activation='relu')(input_layer)
#conv_layer2 = Conv2D(filters=16, kernel_size=(1,1), activation='relu')(conv_layer1)
#conv_layer3 = Conv2D(filters=32, kernel_size=(1,1), activation='relu')(conv_layer2)
#conv_layer4 = Conv2D(filters=64, kernel_size=(1,1), activation='relu')(conv_layer3)

## 3D Convolutional Layers
conv_layer1 = Conv3D(filters=8, kernel_size=(3,3,7), activation='relu')(input_layer)
conv_layer2 = Conv3D(filters=16, kernel_size=(3,3,5), activation='relu')(conv_layer1)
conv_layer3 = Conv3D(filters=32, kernel_size=(3,3,3), activation='relu')(conv_layer2)
conv_layer4 = Conv3D(filters=64, kernel_size=(3,3,3), activation='relu')(conv_layer3)


## Faltten 3D Convolutional Layer
flatten_layer = Flatten()(conv_layer1)#4to1
## Fully Connected Layers
dense_layer1 = Dense(units=256, activation='relu')(flatten_layer)
dense_layer1 = Dropout(0.4)(dense_layer1)
dense_layer2 = Dense(units=128, activation='relu')(dense_layer1)
dense_layer2 = Dropout(0.4)(dense_layer2)
output_layer = Dense(units=16, activation='softmax')(dense_layer2)

# define the model with input layer and output layer
model = Model(inputs=input_layer, outputs=output_layer)
model.summary()
# compiling the model
adam = Adam(lr=0.001, decay=1e-06)#change .001 to .01
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
## Training Time and Fit the Model
start = time.time()
#history = model.fit(x=Tr, y=TrC, batch_size=256, epochs=50, validation_data=(Tv, TvC))#ep=50 t020
#end = time.time()
#Tr_Time = end - start
history = model.fit(x=Tr, y=TrC, batch_size=256, epochs=50, validation_data=(Tv, TvC))  # ep=50 t020
#end = time.time()
#Tr_Time = end - start

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 9, 9, 20, 1)]     0         
                                                                 
 conv3d (Conv3D)             (None, 7, 7, 14, 8)       512       
                                                                 
 flatten_1 (Flatten)         (None, 5488)              0         
                                                                 
 dense_2 (Dense)             (None, 256)               1405184   
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_3 (Dense)             (None, 128)               32896     
                                                                 
 dropout_2 (Dropout)         (None, 128)               0     

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.utils import np_utils

# Load your hyperspectral data and ground truth
#HSI = np.load('your_hsi_data.npy')  # Replace with your data file
#GT = np.load('your_ground_truth.npy')  # Replace with your ground truth file
HSI = scipy.io.loadmat('/content/drive/MyDrive/Colab Notebooks/Indian_pines_corrected.mat')['indian_pines_corrected'];
#GT = labels = sio.loadmat(os.path.join(data_path, 'Indian_pines_gt.mat'))['indian_pines_gt']
GT = scipy.io.loadmat('/content/drive/MyDrive/Colab Notebooks/Indian_pines_gt.mat')['indian_pines_gt']

# Split the data into training and test sets
Tr, Te, TrC, TeC = train_test_split(HSI, GT, test_size=0.2, random_state=42)

# Define the CNN model
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(9, 9, HSI.shape[2])))
model.add(Flatten())
model.add(Dense(units=128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=np.unique(GT).shape[0], activation='softmax'))

# Compile the model
adam = Adam(lr=0.001, decay=1e-06)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

# Reshape and preprocess the data
Tr = Tr.reshape(-1, 9, 9, HSI.shape[2])
TrC = np_utils.to_categorical(TrC)

# Train the model
model.fit(x=Tr, y=TrC, batch_size=64, epochs=20, validation_split=0.2)

# Evaluate the model on the test set
Te = Te.reshape(-1, 9, 9, HSI.shape[2])
TeC = np_utils.to_categorical(TeC)
score = model.evaluate(Te, TeC, batch_size=32)
print("Test Loss:", score[0])
print("Test Accuracy:", score[1])


ValueError: ignored

In [ ]:
import time
import numpy as np
import scipy.io
from sklearn.model_selection import train_test_split
from sklearn.decomposition import IncrementalPCA
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, cohen_kappa_score
from keras.layers import Input, Conv2D, Conv3D, Flatten, Dense, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import np_utils

# Split Training, Validation, and Test Sets
def SplitTr_Te(HSI, GT, TeRatio, randomState=345):
    Tr, Te, TrC, TeC = train_test_split(HSI, GT, test_size=TeRatio, random_state=randomState, stratify=GT)
    return Tr, Te, TrC, TeC

# ... (other functions remain the same)

## Create Image Cubes for Model Building
HSI, GT = HSICubes(HSI, GT, WinSize=13)
HSI.shape, GT.shape

## Split Train and Test sets
Tr, Te, TrC, TeC = SplitTr_Te(HSI, GT, 0.60)
Tr.shape, Te.shape, TrC.shape, TeC.shape

## Split Train and Validation
Tr, Tv, TrC, TvC = SplitTr_Te(Tr, TrC, 0.30)
Tr.shape, Tv.shape, TrC.shape, TvC.shape

## Model Pre requisites
Tr = Tr.reshape(-1, 13, 13, 13, 1)
TrC = np_utils.to_categorical(TrC)
Tv = Tv.reshape(-1, 13, 13, 13, 1)
TvC = np_utils.to_categorical(TvC)
Tr.shape, TrC.shape, Tv.shape, TvC.shape

## Model Structure
## Input layer
input_layer = Input((13, 13, 13, 1))

## 3D Convolutional Layers
conv_layer1 = Conv3D(filters=8, kernel_size=(3, 3, 3), activation='relu')(input_layer)
conv_layer2 = Conv3D(filters=16, kernel_size=(3, 3, 3), activation='relu')(conv_layer1)
conv_layer3 = Conv3D(filters=32, kernel_size=(3, 3, 3), activation='relu')(conv_layer2)
conv_layer4 = Conv3D(filters=64, kernel_size=(3, 3, 3), activation='relu')(conv_layer3)

## Faltten 3D Convolutional Layer
flatten_layer = Flatten()(conv_layer1)

## Fully Connected Layers
dense_layer1 = Dense(units=256, activation='relu')(flatten_layer)
dense_layer1 = Dropout(0.4)(dense_layer1)
dense_layer2 = Dense(units=128, activation='relu')(dense_layer1)
dense_layer2 = Dropout(0.4)(dense_layer2)
output_layer = Dense(units=16, activation='softmax')(dense_layer2)

# Define the model with input layer and output layer
model = Model(inputs=input_layer, outputs=output_layer)
model.summary()

# Compiling the model
adam = Adam(lr=0.001, decay=1e-06)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

## Training Time and Fit the Model
start = time.time()
history = model.fit(x=Tr, y=TrC, batch_size=256, epochs=50, validation_data=(Tv, TvC))
end = time.time()
Tr_Time = end - start


In [ ]:
## Split Training, Validation and Test Sets
def SplitTr_Te(HSI, GT, TeRatio, randomState=345):
    Tr, Te, TrC, TeC = train_test_split(HSI, GT, test_size=TeRatio, random_state=randomState, stratify=GT)
    return Tr, Te, TrC, TeC
def DL_Method(HSI, numComponents = 75):
    RHSI = np.reshape(HSI, (-1, HSI.shape[2]))
    n_batches = 256
    inc_pca = IncrementalPCA(n_components=numComponents)
    for X_batch in np.array_split(RHSI, n_batches):
        inc_pca.partial_fit(X_batch)
    X_ipca = inc_pca.transform(RHSI)
    RHSI = np.reshape(X_ipca, (HSI.shape[0],HSI.shape[1], numComponents))
    return RHSI
## Padding
def ZeroPad(HSI, margin=2):
    NHSI = np.zeros((HSI.shape[0] + 2 * margin, HSI.shape[1] + 2* margin, HSI.shape[2]))
    x_offset = margin
    y_offset = margin
    NHSI[x_offset:HSI.shape[0] + x_offset, y_offset:HSI.shape[1] + y_offset, :] = HSI
    return NHSI
## Spatial Patches in 3D
def HSICubes(HSI, GT, WinSize=6, removeZeroLabels = True):#5t01
    margin = int((WinSize - 1) / 2)
    zeroPaddedX = ZeroPad(HSI, margin=margin)
    # split patches
    patchesData = np.zeros((HSI.shape[0] * HSI.shape[1], WinSize, WinSize, HSI.shape[2]))
    patchesLabels = np.zeros((HSI.shape[0] * HSI.shape[1]))
    patchIndex = 0
    for r in range(margin, zeroPaddedX.shape[0] - margin):
        for c in range(margin, zeroPaddedX.shape[1] - margin):
            patch = zeroPaddedX[r - margin:r + margin + 1, c - margin:c + margin + 1]
            patchesData[patchIndex, :, :, :] = patch
            patchesLabels[patchIndex] = GT[r-margin, c-margin]
            patchIndex = patchIndex + 1
    if removeZeroLabels:
        patchesData = patchesData[patchesLabels>0,:,:,:]
        patchesLabels = patchesLabels[patchesLabels>0]
        patchesLabels -= 1
    return patchesData, patchesLabels
## Compute Per Class Accuacy form Confusion Matrix
def AA_andEachClassAccuracy(confusion_matrix):
    counter = confusion_matrix.shape[0]
    list_diag = np.diag(confusion_matrix)
    list_raw_sum = np.sum(confusion_matrix, axis=1)
    each_acc = np.nan_to_num(truediv(list_diag, list_raw_sum))
    average_acc = np.mean(each_acc)
    return each_acc, average_acc
## Assigning Class Labels and Computing the Test Accuracy
def reports(Te,TeC):
    start = time.time()
    Y_pred = model.predict(Te)
    y_pred = np.argmax(Y_pred, axis=1)
    end = time.time()
    total = end - start
    target_names = ['Alfalfa', 'Corn-notill', 'Corn-mintill', 'Corn'
                        ,'Grass-pasture', 'Grass-trees', 'Grass-pasture-mowed',
                        'Hay-windrowed', 'Oats', 'Soybean-notill', 'Soybean-mintill',
                        'Soybean-clean', 'Wheat', 'Woods', 'Buildings-Grass-Trees-Drives',
                        'Stone-Steel-Towers']
    classification = classification_report(np.argmax(TeC, axis=1), y_pred, target_names=target_names)
    oa = accuracy_score(np.argmax(TeC, axis=1), y_pred)
    confusion = confusion_matrix(np.argmax(TeC, axis=1), y_pred)
    each_acc, aa = AA_andEachClassAccuracy(confusion)
    kappa = cohen_kappa_score(np.argmax(TeC, axis=1), y_pred)
    score = model.evaluate(Te, TeC, batch_size=32)
    Test_Loss =  score[0]*100
    Test_accuracy = score[1]*100
    return classification, confusion, Test_Loss, Test_accuracy, oa*100, each_acc*100, aa*100, kappa*100, target_names, y_pred, total
## Compute the Patch to Prepare for Ground Truths
def Patch(data,height_index,width_index):
    height_slice = slice(height_index, height_index+PATCH_SIZE)
    width_slice = slice(width_index, width_index+PATCH_SIZE)
    patch = data[height_slice, width_slice, :]
    return patch
## Loading Dataset
import scipy.io
## There is a typo in the paper on page#4 Table#4, the training samples are not 10%, the reported accuracies are obtained with almost 30~40% training samples.
#data_path = os.path.join(os.getcwd(),'/content/drive/My Drive/Colab Notebooks')
#HSI = sio.loadmat(os.path.join(data_path, 'Indian_pines_corrected.mat'))['indian_pines_corrected']
HSI = scipy.io.loadmat('/content/drive/MyDrive/Colab Notebooks/Indian_pines_corrected.mat')['indian_pines_corrected'];
#GT = labels = sio.loadmat(os.path.join(data_path, 'Indian_pines_gt.mat'))['indian_pines_gt']
GT = scipy.io.loadmat('/content/drive/MyDrive/Colab Notebooks/Indian_pines_gt.mat')['indian_pines_gt']
HSI.shape, GT.shape
## Reduce the Dimensionality
HSI = DL_Method(HSI, numComponents=20)
HSI.shape
## Create Image Cubes for Model Building
HSI, GT = HSICubes(HSI, GT, WinSize=13)
HSI.shape, GT.shape
## Split Train and Test sets
Tr, Te, TrC, TeC = SplitTr_Te(HSI, GT, 0.60)
Tr.shape, Te.shape, TrC.shape, TeC.shape
## Split Train and Validation
Tr, Tv, TrC, TvC = SplitTr_Te(Tr, TrC, 0.40)
Tr.shape, Tv.shape, TrC.shape, TvC.shape
## Model Pre requsites
Tr = Tr.reshape(-1, 13, 13, 20, 1)
TrC = np_utils.to_categorical(TrC)
Tv = Tv.reshape(-1, 13, 13, 20, 1)
TvC = np_utils.to_categorical(TvC)
Tr.shape, TrC.shape, Tv.shape, TvC.shape

## Model Structure
## Input layer
input_layer = Input((13, 13, 20, 1))
## 2D Convolutional Layers
conv_layer1 = Conv2D(filters=8, kernel_size=(1,1), activation='relu')(input_layer)
conv_layer2 = Conv2D(filters=16, kernel_size=(1,1), activation='relu')(conv_layer1)
conv_layer3 = Conv2D(filters=32, kernel_size=(1,1), activation='relu')(conv_layer2)
conv_layer4 = Conv2D(filters=64, kernel_size=(1,1), activation='relu')(conv_layer3)

## 3D Convolutional Layers
conv_layer1 = Conv3D(filters=8, kernel_size=(3,3,7), activation='relu')(input_layer)
conv_layer2 = Conv3D(filters=16, kernel_size=(3,3,5), activation='relu')(conv_layer1)
conv_layer3 = Conv3D(filters=32, kernel_size=(3,3,3), activation='relu')(conv_layer2)
conv_layer4 = Conv3D(filters=64, kernel_size=(3,3,3), activation='relu')(conv_layer3)


## Faltten 3D Convolutional Layer
flatten_layer = Flatten()(conv_layer1)#4to1
## Fully Connected Layers
dense_layer1 = Dense(units=256, activation='relu')(flatten_layer)
dense_layer1 = Dropout(0.4)(dense_layer1)
dense_layer2 = Dense(units=128, activation='relu')(dense_layer1)
dense_layer2 = Dropout(0.4)(dense_layer2)
output_layer = Dense(units=16, activation='softmax')(dense_layer2)

# define the model with input layer and output layer
model = Model(inputs=input_layer, outputs=output_layer)
model.summary()
# compiling the model
adam = Adam(lr=0.001, decay=1e-06)#change .001 to .01
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
## Training Time and Fit the Model
start = time.time()
#history = model.fit(x=Tr, y=TrC, batch_size=256, epochs=50, validation_data=(Tv, TvC))#ep=50 t020
#end = time.time()
#Tr_Time = end - start
history = model.fit(x=Tr, y=TrC, batch_size=256, epochs=50, validation_data=(Tv, TvC))  # ep=50 t020
#end = time.time()
#Tr_Time = end - start